In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})
import pickle
from scipy.optimize import curve_fit
from my_functions import schechter, double_schechter, count_true

In [ ]:
filename = '/home/alberto/cosmos/LAEs/Luminosity_functions/LF_r17-24_z2.5-3.8_ew0_ewoth400_nb/LFs.pkl'
with open(filename, 'rb') as f:
    LFdic = pickle.load(f)

LF_phi = LFdic['LF_total']
LF_bin = LFdic['LF_bins']
LF_yerr_minus = LFdic['LF_total_err'][0]
LF_yerr_plus = LFdic['LF_total_err'][0]
LF_xerr = LFdic['LF_total_err'][0]

In [ ]:
def sch_fit(Lx, Phistar, Lstar, alpha):
    return schechter(Lx, Phistar, Lstar, alpha) * Lx * np.log(10)

In [ ]:
where_fit = (LF_bin > 43.5) & (LF_phi > 0)

sigma = (LF_yerr_minus + LF_yerr_plus) * 0.5

popt, pcov = curve_fit(sch_fit, 10**LF_bin[where_fit], LF_phi[where_fit],
                       p0=[1e-6, 1e44, -1.5], sigma=sigma[where_fit])

In [ ]:
Lx = np.logspace(42, 46, 1000)

N_fit_bins = count_true(where_fit)
N_iter = 100
fit_mat = np.empty((0, len(Lx)))  # Matrix with the params of all iterations
LF_i_bin = 10 ** LF_bin[where_fit]
for i in range(N_iter):
    randN = np.random.normal(size=(N_fit_bins,))
    LF_i_phi = LF_phi[where_fit] + randN * sigma[where_fit]
    try:
        popt_aux, _ = curve_fit(sch_fit, LF_i_bin, LF_i_phi,
                                p0=[1e-6, 1e44, -1.5])
    except:
        continue
    Phi_aux = sch_fit(Lx, *tuple(popt_aux))
    fit_mat = np.vstack([fit_mat, Phi_aux])

Phi16, Phi50, Phi84 = np.percentile(fit_mat, [16, 50, 48], axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

# Plot the reference LF curves

phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = double_schechter(
    Lx, phistar1, 10 ** Lstar1, alpha1, 10 ** phistar2, 10 ** Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=1, color='C6'
)

phistar1 = 10 ** -3.41
Lstar1 = 10 ** 42.87
alpha1 = -1.7

phistar2 = 10 ** -5.85
Lstar2 = 10 ** 44.6
alpha2 = -1.2

Phi_center = double_schechter(
    Lx, phistar1, Lstar1, alpha1, phistar2, Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    zorder=0, color='C7'
)

Phi = sch_fit(Lx, *tuple(popt))
ax.plot(np.log10(Lx), Phi, label='My fit')

ax.errorbar(LF_bin[where_fit], LF_phi[where_fit],
            yerr=[LF_yerr_minus[where_fit], LF_yerr_plus[where_fit]],
            xerr=LF_xerr[where_fit],
            fmt='s', color='r', capsize=4, label='My points')

ax.set_yscale('log')
ax.set_ylim(1e-8, 5e-3)
ax.set_xlim(42.5, 45.5)
ax.legend()

Phistar = popt[0]
Lstar = np.log10(popt[1])
alpha = popt[2]
Phistar_err = pcov[0, 0] ** 0.5
Lstar_err = -np.log10((10 ** Lstar - pcov[1, 1] ** 0.5)) + Lstar
alpha_err = pcov[2, 2] ** 0.5

text = fr'''
$\Phi^*=${Phistar:0.1e}$\pm${Phistar_err:0.1e}
$L^*=${Lstar:0.2f}$\pm${Lstar_err:0.2f}
$\alpha=${alpha:0.2f}$\pm${alpha_err:0.2f}
'''
ax.text(42.7, 1e-7, text)

plt.show()

In [ ]:
print(popt)
print()
print(pcov)

In [ ]:
path = '/home/alberto/cosmos/LAEs/Luminosity_functions/LF_r17-24_z2.5-3.8_ew0_ewoth400_nb'
filename = f'{path}/Sch_fit.npy'
schfit = np.load(filename)